### CNN for time series forecasting
src: https://machinelearningmastery.com/how-to-develop-convolutional-neural-network-models-for-time-series-forecasting/

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [9]:
data = pd.read_csv('data/GOOG_040819 - 200416.csv', date_parser=True)

In [10]:
data = data.drop(['Adj Close'], axis=1)

In [11]:
data

,Date,Open,High,Low,Close,Volume
0,2004-08-19,49.813286,51.835709,47.800831,49.982655,44871300
1,2004-08-20,50.316402,54.336334,50.062355,53.952770,22942800
2,2004-08-23,55.168217,56.528118,54.321388,54.495735,18342800
3,2004-08-24,55.412300,55.591629,51.591621,52.239193,15319700
4,2004-08-25,52.284027,53.798351,51.746044,52.802086,9232100
...,...,...,...,...,...,...
3936,2020-04-08,1206.500000,1219.069946,1188.160034,1210.280029,1975100
3937,2020-04-09,1224.079956,1225.569946,1196.734985,1211.449951,2175400
3938,2020-04-13,1209.180054,1220.510010,1187.598022,1217.560059,1739800
3939,2020-04-14,1245.089966,1282.069946,1236.930054,1269.229980,2470400


#### 1) Univariate CNN Models
The CNN model will learn a function that maps a sequence of past observations as input to an output observation. 

In [13]:
univariate_sequence = data['Close'].copy()

In [22]:
close = univariate_sequence

In [25]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [27]:
# choose a number of time steps
n_steps = 8
# split into samples
X, y = split_sequence(close, n_steps)
# summarize the data
for i in range(len(X)):
    print(X[i], y[i])
    if i > 4:
        break

[49.982655 53.95277  54.495735 52.239193 52.802086 53.753517 52.876804
 50.814533] 50.993862
[53.95277  54.495735 52.239193 52.802086 53.753517 52.876804 50.814533
 50.993862] 49.93782
[54.495735 52.239193 52.802086 53.753517 52.876804 50.814533 50.993862
 49.93782 ] 50.565468
[52.239193 52.802086 53.753517 52.876804 50.814533 50.993862 49.93782
 50.565468] 49.818267999999996
[52.802086 53.753517 52.876804 50.814533 50.993862 49.93782  50.565468
 49.818268] 50.600338
[53.753517 52.876804 50.814533 50.993862 49.93782  50.565468 49.818268
 50.600338] 50.958991999999995


In [28]:
X.shape, y.shape

((3933, 8), (3933,))

#### 1D CNN
A one-dimensional CNN is a CNN model that has a convolutional hidden layer that operates over a 1D sequence.

In [29]:
# univariate cnn example
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
print(X.shape)

(3933, 8, 1)


Using TensorFlow backend.


In [31]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 7, 64)             192       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 3, 64)             0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 192)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                9650      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 51        
Total params: 9,893
Trainable params: 9,893
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=50, verbose=0, batch_size=32)

In [43]:
test = X[-1:]
test

array([[[1120.839966],
        [1097.880005],
        [1186.920044],
        [1186.51001 ],
        [1210.280029],
        [1211.449951],
        [1217.560059],
        [1269.22998 ]]])

In [41]:
test.shape

(1, 8, 1)

In [42]:
# demonstrate prediction
x_input = test
# x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1239.2535]]


#### 1) Multivariate One-Step CNN Models
Multivariate time series data means data where there is more than one observation for each time step.

In [47]:
# multivariate data preparation
from numpy import array
from numpy import hstack

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))

# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]

# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [48]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, 2, 64)             320       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 51        
Total params: 3,621
Trainable params: 3,621
Non-trainable params: 0
_________________________________________________________________


Conv1D: (None, 2, 64)

2 -> input was 3, kernel size 2, strides=1 -> results in size 2

64 -> because there are 64 filters

320 params -> note that 1 kernel has 4 parameters now: it has size 2, there are two input variables, therefore 2 * 2. So 64 filters (kernels) * 4 + 64 (biases) = 320 params

In [50]:
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=1000, verbose=0)
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[205.71127]]


#### 2) Univariate Multi-Step CNN Models (Vector Output)
1D CNN model predicts a vector output that represents multiple time steps of one variable.

In [54]:
# univariate multi-step vector-output 1d cnn example
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps_in, n_steps_out = 3, 2
# split into samples
X, y = split_sequence(raw_seq, n_steps_in, n_steps_out)
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
X.shape, y.shape

((5, 3, 1), (5, 2))

In [53]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(n_steps_out))

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 102       
Total params: 3,544
Trainable params: 3,544
Non-trainable params: 0
_________________________________________________________________


In [55]:
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps_in, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[103.22777 115.58105]]


#### 3) Multivariate Multi-Step CNN Models
Develop a 1D CNN with Multiple Input Multi-Step Output.